# Binomial Classification

In [ ]:
case class cancer(x1: Long, y: Long)

val trainData = spark.createDataFrame(Seq(cancer(330, 1), cancer(120, 0), cancer(400, 1))).toDF
val testData = spark.createDataFrame(Seq(cancer(500, 0))).toDF

In [ ]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("x1")).setOutputCol("features")

val train = va.transform(trainData)
val test = va.transform(testData)

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression()
    .setFeaturesCol("features")
    .setLabelCol("y")
    .setMaxIter(10)
    .setRegParam(0.3)
    .setElasticNetParam(0.8)

val lrModel = lr.fit(train)

lrModel.transform(test).show

# Multi-Class Classification

In [ ]:
val training = spark.read.format("libsvm").load("multiclass_data.txt")

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)
val lrModel = lr.fit(training)

In [ ]:
println(s"Coefficients: \n${lrModel.coefficientMatrix}")
println(s"Intercepts: \n${lrModel.interceptVector}")

# Binomial Logistic Regression Measurements

In [ ]:
import org.apache.spark.ml.feature.VectorAssembler

case class cancer(x1: Long, label: Long)

val trainData = spark.createDataFrame(Seq(cancer(330, 1), cancer(120, 0), cancer(400, 1))).toDF
val va = new VectorAssembler().setInputCols(Array("x1")).setOutputCol("features")
val train = va.transform(trainData)

In [ ]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.sql.functions._


val lr = new LogisticRegression()
val lrModel = lr.fit(train)

val trainingSummary = lrModel.binarySummary

// obtain the objective per iteration.
val objectiveHistory = trainingSummary.objectiveHistory
objectiveHistory.foreach(loss => println(loss))

// obtain the ROC as a dataframe and areaUnderROC.
val roc = trainingSummary.roc
roc.show()
println(s"areaUnderROC: ${trainingSummary.areaUnderROC}")

// set the model threshold to maximize F-Measure
val fMeasure = trainingSummary.fMeasureByThreshold
val maxFMeasure = fMeasure.select(max("F-Measure")).head().getDouble(0)
val bestThreshold = fMeasure.where($"F-Measure" === maxFMeasure).select("threshold").head().getDouble(0)
lrModel.setThreshold(bestThreshold)